<a href="https://colab.research.google.com/github/shulifinley/poly-or-rand/blob/main/galois_rank_parameterized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install galois

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 28.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import galois
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
%config InteractiveShell.ast_node_interactivity='all'

from sklearn.metrics import mean_squared_error


In [ ]:
galois.__version__

'0.3.5'

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# function to convert function evaluations to flattened one-hot encoding vector
def evals_to_one_hot(GF, e_array, evals):
  q = GF.characteristic
  eval_one_hot = GF.Zeros((q, q))
  eval_one_hot[e_array, evals] = 1
  eval_one_hot = eval_one_hot.flatten()
  return eval_one_hot

In [ ]:
q = 7
d = 0
GF = galois.GF(q**1)
e_array = GF.Range(0,q)
pi = np.arange(q**2)
np.random.shuffle(pi)
print(pi)


[46 41 11 16 17 13 19 24 25 27  4 36  3  7  5 15 31 34  9 47 22 39 23 37
 32 44 28 29 26 48 38 40  8  2  0 33 43 14 30 12  6 20  1 18 45 10 35 42
 21]


In [ ]:
poly_array = np.array([galois.Poly.Random(degree=d, field=GF) for i in range(q**2)])
# print(poly_array)
evals_array = np.array([evals_to_one_hot(GF, e_array, f(e_array)).flatten() for f in poly_array])#.flatten()
# print(evals_array)
evals_array = np.array([list(map(eval.__getitem__, pi)) for eval in evals_array])
print()
# print(evals_array)
print()
# evals_array = evals_array.flatten()
print(evals_array)
D_poly = GF(np.array(evals_array))
print(type(D_poly), D_poly)
print(D_poly.shape)
np.linalg.matrix_rank(D_poly)



[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'galois.GF(7)'> [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(49, 49)


6

In [ ]:
def calc_rank_upper_bound(q, d):
  return (q**2)-(2*q)+d+2

def calc_d_lower_bound(q, rank):
  return rank-(q**2)+(2*q)-2

In [ ]:
def rank_experiment(q, ds, num_instances, **kwargs):
  GF = galois.GF(q**1)

  # array of field elements
  e_array = GF.Range(0,q)

  poly_dict = {}

  print("q: " + str(q) + ", number of instances per degree: " + str(num_instances))
  print()

  dict_describe = {str(key): {} for key in ds}

  for d in ds:
    dict_describe[str(d)]['rank_bound'] = calc_rank_upper_bound(q, d)
    poly_dict[str(d)] = {}
    computed_rank_list = []
    for i in range(num_instances):
      poly_array = np.array([galois.Poly.Random(degree=d, field=GF) for i in range(q**2)])
      evals_array = np.array([evals_to_one_hot(GF, e_array, f(e_array)).flatten() for f in poly_array])
      D_poly = GF(np.array(evals_array))
      rank = np.linalg.matrix_rank(D_poly)
      computed_rank_list.append(rank)

    computed_rank_arr = np.array(computed_rank_list)
    dict_describe[str(d)]['computed_ranks'], dict_describe[str(d)]['counts'] = np.unique(computed_rank_arr, return_counts=True)
    dict_describe[str(d)]['computed_d_bounds'] = (dict_describe[str(d)]['computed_ranks']) - ((q**2) + (2*q) - 2)

    df_describe = pd.DataFrame(dict_describe[str(d)])
    df_describe = df_describe[['computed_ranks', 'computed_d_bounds', 'counts']]
    df_describe['rank_bound_delta'] = abs(df_describe['computed_ranks'] - dict_describe[str(d)]['rank_bound'])
    df_describe['d_bound_delta'] = abs(df_describe['computed_d_bounds'] - d)

    rank_acc_rate = (df_describe.loc[df_describe['computed_ranks']==dict_describe[str(d)]['rank_bound'], 'counts']).sum()
    dict_describe[str(d)]['rank_acc_rate'] = (rank_acc_rate/num_instances)
    # dict_describe[str(d)]['rank_mse'] = np.array([(computed - dict_describe[str(d)]['rank_bound'])**2 for computed in df_describe['computed_ranks']])
    # dict_describe[str(d)]['rank_mse'] = np.sum(dict_describe[str(d)]['rank_mse'] * dict_describe[str(d)]['counts']) / num_instances

    rank_mse = mean_squared_error(df_describe['computed_ranks'], [dict_describe[str(d)]['rank_bound']]*len(df_describe['computed_ranks']), sample_weight=dict_describe[str(d)]['counts'], squared=True)
    dict_describe[str(d)]['rank_mse'] = rank_mse

    print('Computed ranks and d lower bounds for (q = {}, d = {})'.format(q,d))
    print('Rank(D) upper bound = {}'.format(dict_describe[str(d)]['rank_bound']))
    print('Percentage of instances for which (computed Rank(D) = Rank(D) bound): {}%'.format(dict_describe[str(d)]['rank_acc_rate']*100))
    print('Rank MSE (using rank bound as true label): {}'.format(dict_describe[str(d)]['rank_mse']))

    print(df_describe)

    print()


  return dict_describe



In [ ]:
q3_results = rank_experiment(q=3, ds=[0, 1, 2, 3], num_instances=100)

q: 3, number of instances per degree: 100

Computed ranks and d lower bounds for (q = 3, d = 0)
Rank(D) upper bound = 5
Percentage of instances for which (computed Rank(D) = Rank(D) bound): 0.0%
Rank MSE (using rank bound as true label): 9.0
   computed_ranks  computed_d_bounds  counts  rank_bound_delta  d_bound_delta
0               2                -11     100                 3             11

Computed ranks and d lower bounds for (q = 3, d = 1)
Rank(D) upper bound = 6
Percentage of instances for which (computed Rank(D) = Rank(D) bound): 0.0%
Rank MSE (using rank bound as true label): 1.99
   computed_ranks  computed_d_bounds  counts  rank_bound_delta  d_bound_delta
0               3                -10       3                 3             11
1               4                 -9      25                 2             10
2               5                 -8      72                 1              9

Computed ranks and d lower bounds for (q = 3, d = 2)
Rank(D) upper bound = 7
Percentage 

In [ ]:
k = 1000

In [ ]:
q3_results = rank_experiment(q=3, ds=[0, 1, 2, 3], num_instances=k)


q: 3, number of instances per degree: 1000

Computed ranks and d lower bounds for (q = 3, d = 0)
Rank(D) upper bound = 5
Percentage of instances for which (computed Rank(D) = Rank(D) bound): 0.0%
Rank MSE (using rank bound as true label): 9.049
   computed_ranks  computed_d_bounds  counts  rank_bound_delta  d_bound_delta
0               1                -12       7                 4             12
1               2                -11     993                 3             11

Computed ranks and d lower bounds for (q = 3, d = 1)
Rank(D) upper bound = 6
Percentage of instances for which (computed Rank(D) = Rank(D) bound): 0.0%
Rank MSE (using rank bound as true label): 2.181
   computed_ranks  computed_d_bounds  counts  rank_bound_delta  d_bound_delta
0               2                -11       4                 4             12
1               3                -10      40                 3             11
2               4                 -9     267                 2             10
3      

In [ ]:
q7_results = rank_experiment(q=7, ds=[0, 1, 2, 3 ], num_instances=k)



q: 7, number of instances per degree: 1000

Computed ranks and d lower bounds for (q = 7, d = 0)
Rank(D) upper bound = 37
Percentage of instances for which (computed Rank(D) = Rank(D) bound): 0.0%
Rank MSE (using rank bound as true label): 961.063
   computed_ranks  computed_d_bounds  counts  rank_bound_delta  d_bound_delta
0               5                -56       1                32             56
1               6                -55     999                31             55

Computed ranks and d lower bounds for (q = 7, d = 1)
Rank(D) upper bound = 38
Percentage of instances for which (computed Rank(D) = Rank(D) bound): 0.0%
Rank MSE (using rank bound as true label): 126.408
   computed_ranks  computed_d_bounds  counts  rank_bound_delta  d_bound_delta
0              22                -39       2                16             40
1              23                -38       3                15             39
2              24                -37      12                14             38
3

In [ ]:
# TODO: permute!!!
# q7_results = rank_experiment(q=7, ds=[0, 1, 2, 3 ], num_instances=k)


In [ ]:

q17_results = rank_experiment(q=17, ds=[0, 1, 2, 3, 5, 8], num_instances=k)


q: 17, number of instances per degree: 1000

Computed ranks and d lower bounds for (q = 17, d = 0)
Rank(D) upper bound = 257
Percentage of instances for which (computed Rank(D) = Rank(D) bound): 0.0%
Rank MSE (using rank bound as true label): 58081.0
   computed_ranks  computed_d_bounds  counts  rank_bound_delta  d_bound_delta
0              16               -305    1000               241            305

Computed ranks and d lower bounds for (q = 17, d = 1)
Rank(D) upper bound = 258
Percentage of instances for which (computed Rank(D) = Rank(D) bound): 0.0%
Rank MSE (using rank bound as true label): 11236.0
   computed_ranks  computed_d_bounds  counts  rank_bound_delta  d_bound_delta
0             152               -169    1000               106            170

Computed ranks and d lower bounds for (q = 17, d = 2)
Rank(D) upper bound = 259
Percentage of instances for which (computed Rank(D) = Rank(D) bound): 0.0%
Rank MSE (using rank bound as true label): 1764.0
   computed_ranks  compu

In [ ]:
q=23
ds=[0,1,2,3,5,8,10]
k=100
q23_results = rank_experiment(q=q, ds=ds, num_instances=k)


q: 23, number of instances per degree: 100

Computed ranks and d lower bounds for (q = 23, d = 0)
Rank(D) upper bound = 485
Percentage of instances for which (computed Rank(D) = Rank(D) bound): 0.0%
Rank MSE (using rank bound as true label): 214369.0
   computed_ranks  computed_d_bounds  counts  rank_bound_delta  d_bound_delta
0              22               -551     100               463            551

Computed ranks and d lower bounds for (q = 23, d = 1)
Rank(D) upper bound = 486
Percentage of instances for which (computed Rank(D) = Rank(D) bound): 0.0%
Rank MSE (using rank bound as true label): 44521.0
   computed_ranks  computed_d_bounds  counts  rank_bound_delta  d_bound_delta
0             275               -298     100               211            299

Computed ranks and d lower bounds for (q = 23, d = 2)
Rank(D) upper bound = 487
Percentage of instances for which (computed Rank(D) = Rank(D) bound): 0.0%
Rank MSE (using rank bound as true label): 8100.0
   computed_ranks  compu

In [ ]:
q=67
ds=[0,5,10,15,33]
k=100
q67_results = rank_experiment(q=q, ds=ds, num_instances=k)

q: 67, number of instances per degree: 100

Computed ranks and d lower bounds for (q = 67, d = 0)
Rank(D) upper bound = 4357.
Percentage of instances for which computed Rank(D) = Rank(D) bound.
   computed_ranks  computed_d_bounds  counts  rank_bound_delta  d_bound_delta
0              66              -4555     100              4291           4555



KeyboardInterrupt: ignored

In [ ]:
poly_results_23 = rank_experiment(q=23, ds=[1, 3, 7, 12, 22, 23], num_instances=k)


q: 23, number of instances per degree: 1000

Our rank(D) upper bound for d=1: 486
empirical (rank, count): 
[(275, 1000)]

Our rank(D) upper bound for d=3: 488
empirical (rank, count): 
[(437, 1000)]

Our rank(D) upper bound for d=3: 488
empirical (rank, count): 
[(437, 1000)]

Our rank(D) upper bound for d=3: 488
empirical (rank, count): 
[(437, 1000)]

Our rank(D) upper bound for d=7: 492
empirical (rank, count): 
[(483, 1000)]

Our rank(D) upper bound for d=7: 492
empirical (rank, count): 
[(483, 1000)]

Our rank(D) upper bound for d=7: 492
empirical (rank, count): 
[(483, 1000)]

Our rank(D) upper bound for d=12: 497
empirical (rank, count): 
[(497, 1000)]

Our rank(D) upper bound for d=12: 497
empirical (rank, count): 
[(497, 1000)]

Our rank(D) upper bound for d=12: 497
empirical (rank, count): 
[(497, 1000)]

Our rank(D) upper bound for d=22: 507
empirical (rank, count): 
[(507, 1000)]

Our rank(D) upper bound for d=22: 507
empirical (rank, count): 
[(507, 1000)]

Our rank(D) up

KeyboardInterrupt: ignored

KeyboardInterrupt: ignored

KeyboardInterrupt: ignored

In [ ]:
poly_results_67 = rank_experiment(q=67, ds=[1, 3, 7, 12, 22, 66], num_instances=k)


q: 67, number of instances per degree: 1000



In [ ]:
# pi_23 = np.arange(23**2)
# np.random.shuffle(pi_23)

# poly_results_23 = rank_experiment(q=23, ds=[1, 2, 3, 7, 12, 22, 23], num_instances=k, pi=pi_23)


In [ ]:
poly_results_17 = rank_experiment(q=17, ds=[1, 3, 7, 12, 16, 17], num_instances=k)


q: 17, number of instances per degree: 100

Our rank(D) bound for d=1: 258
empirical (rank, count): 
[(152, 100)]

Our rank(D) bound for d=3: 260
empirical (rank, count): 
[(238, 100)]

Our rank(D) bound for d=7: 264
empirical (rank, count): 
[(262, 100)]

Our rank(D) bound for d=12: 269
empirical (rank, count): 
[(269, 100)]

Our rank(D) bound for d=16: 273
empirical (rank, count): 
[(273, 100)]

Our rank(D) bound for d=17: 274
empirical (rank, count): 
[(273, 100)]



In [ ]:
# q = 17
# pi = np.arange(q**2)
# np.random.shuffle(pi)

# poly_results_17, random_binary_results_17 = rank_experiment(q=q, ds=[4, 8, 12, 17], num_instances = k)

In [ ]:
# q = 19
# pi = np.arange(q**2)
# np.random.shuffle(pi)

# poly_results_19, random_binary_results_19 = rank_experiment(q=q, ds=[4, 8, 12, 17, 19], num_instances = k)

In [ ]:
# q = 23
# pi = np.arange(q**2)
# np.random.shuffle(pi)

# poly_results_23, random_binary_results_23 = rank_experiment(q=q, ds=[4, 8, 12, 17, 19, 23], num_instances = k)

In [ ]:
import pprint

In [ ]:
random_ranks_17, random_ranks_17_count = np.unique(random_binary_results_17['rank_list'], return_counts=True)

print('GF(17)')
print()
pprint.pprint(poly_results_17)
print()
print('Random binary matrix')
print(random_ranks_17, random_ranks_17_count)


In [ ]:
plt.hist((poly_results_17['4']['rank_list'], poly_results_17['4']['rank_count']))
plt.hist((poly_results_17['8']['rank_list'], poly_results_17['8']['rank_count']))
plt.hist((poly_results_17['12']['rank_list'], poly_results_17['12']['rank_count']))
plt.hist((poly_results_17['17']['rank_list'], poly_results_17['17']['rank_count']))


In [ ]:
random_ranks_19, random_ranks_19_count = np.unique(random_binary_results_19['rank_list'], return_counts=True)

print('GF(19)')
print()
pprint.pprint(poly_results_19)
print()
print('Random binary matrix')
print(random_ranks_19, random_ranks_19_count)

In [ ]:
random_ranks_23, random_ranks_23_count = np.unique(random_binary_results_23['rank_list'], return_counts=True)
dim = q**2
print('GF(23), 23^2= ', dim)
print()
pprint.pprint(poly_results_23)
print()
print('Random binary matrix')
print(random_ranks_23, random_ranks_23_count)

In [ ]:
print("GF(23):")
low_ranks, low_ranks_count = np.unique(low_deg_dict['rank_list'], return_counts=True)
print("100 intances of d=4 ranks: {0}, counts: {1}".format(low_ranks, low_ranks_count))

rand_ranks, rand_ranks_count = np.unique(rand_deg_dict['rank_list'], return_counts=True)
print("100 intances of d=23 ranks: {0}, counts: {1}".format(rand_ranks, rand_ranks_count))

rand_bin_ranks, rand_bin_ranks_count = np.unique(rand_binary_dict['rank_list'], return_counts=True)
print("100 intances of random binary ranks: {0}, counts: {1}".format(rand_bin_ranks, rand_bin_ranks_count))



In [ ]:
# array of q**2=19**2=361 polynomials of degree d=4
poly_array = np.array([galois.Poly.Random(degree=d, field=GF) for i in range(q**2)])
poly_array.shape

In [ ]:
# matrix representation of all q^2 polynomials
D = np.array([evals_to_column(f(e_array)).flatten() for f in poly_array])
D = GF(D)
D

In [ ]:
D[0], D[0].shape

In [ ]:
D.shape, q**2, D.size, q**4

In [ ]:
np.linalg.det(D)

In [ ]:
np.linalg.matrix_rank(D)